In [157]:
### generate complex data 
import numpy as np
np.random.seed(5)

def generate_data(n_samples, true_coefficients, noise_level=0.1):

    # Generate a random input signal
    k = len(true_coefficients)
    m, n = true_coefficients[0].shape


    x = np.random.randn(n_samples, m) + 1j*np.random.randn(n_samples, m)

    # Generate the desired signal by passing the input through the known system
    y = np.zeros((n_samples, n), dtype=np.complex128)

    ### y is complex data
    for j in range(k):
        y += np.dot(np.roll(x, j, axis=0), true_coefficients[j])

    # Add noise
    noise = noise_level * np.random.randn(n_samples, n) + 1j*noise_level * np.random.randn(n_samples, n)
    y += noise
    
    return x, y

# Example usage
n_samples = 30000
# a series of matrix: k x m x n 
# k : influence scope 5
# m : output dimension  3
# n : input dimension 2

k = 2
m = 3
n = 2
## coef is complex 
true_coefficients = [np.random.randn(m, n) + 1j*np.random.randn(m, n) for _ in range(k)]

noise_level = 0.05
x, y = generate_data(n_samples, true_coefficients, noise_level)
print("Input signal:", x[:3])
print("Desired signal:", y[:3])

Input signal: [[ 0.99643983+1.22619166j  0.71242127-1.2122101j   0.05914424+0.39883307j]
 [-0.36331088-0.40809211j  0.00328884-0.43437028j -0.10593044+1.37118283j]
 [ 0.79305332-0.56804962j -0.63157163-2.03992636j -0.00619491-0.3224276j ]]
Desired signal: [[10.15713696-7.99924289j  5.86412312+2.20694763j]
 [ 0.68749342-0.83251454j -1.64045422+5.04276785j]
 [-1.60766596-3.1046495j  -1.14405067+2.31378932j]]


In [158]:
### LMS filter
k = len(true_coefficients)
def lms_filter(x, y, k, mu ):
    Ws = [] 
    m, n = x.shape[1], y.shape[1]
    w = np.array([np.random.randn(m, n) + 1j*np.random.randn(m, n) for _ in range(k)]) ## k x m x n
    e = np.zeros((len(x), n), dtype=complex) ## record error

    for i in range(k, len(x)):
        y_hat = np.zeros((n), dtype=complex)
        for j in range(k):
            y_hat += np.dot(x[i-j], w[j])
        error = y[i] - y_hat 
        e[i] = error
        for j in range(k):
            ### update w[j], using wirtinger derivative
            w[j] += mu * np.outer(np.conj(x[i-j]) , error)
        Ws.append(w.reshape(k, m, n).copy())
    return w,e, Ws


In [164]:
w,e, Ws = lms_filter(x, y, k, 0.05)
print("Estimated coefficients:", w[1])
print ("True coefficients:", true_coefficients[1])

Estimated coefficients: [[-0.36501639-1.50914096j  0.59478804+0.64064571j]
 [-1.66380618-0.97273044j -0.69797699-0.86363545j]
 [ 1.13673211-0.87572423j  1.86862167-0.43693172j]]
True coefficients: [[-0.35882895-1.51117956j  0.6034716 +0.64484751j]
 [-1.66478853-0.98060789j -0.70017904-0.85685315j]
 [ 1.15139101-0.87187918j  1.85733101-0.42250793j]]


In [160]:
y[4]

array([ 5.45087576-0.99003013j, -0.76182188+0.36619373j])

In [161]:
print( np.dot(x[4], true_coefficients[0]) ) 

[ 2.51854698+0.13627228j -1.0545789 -0.06986256j]


In [162]:
Ws[300]

array([[[ 0.43650742-0.89897926j, -0.32068658-0.57703637j],
        [ 2.41565879+0.19618413j, -0.26626511-0.333288j  ],
        [ 0.10014935-1.2030087j ,  1.57368391-0.20778811j]],

       [[-0.35376631-1.50558865j,  0.60979032+0.64189298j],
        [-1.67154166-0.99366809j, -0.68560852-0.83954914j],
        [ 1.15058299-0.87669545j,  1.83168272-0.4162798j ]]])

In [163]:
true_coefficients[0].shape

(3, 2)